In [ ]:
"""
Created on Fri Aug 09 11:56 2024

This script is to make 2D maps of hydrofracturing limits

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
GL_flux_ag = xr.Dataset()
GL_flux_ag['flux_Gt_ref'] = xr.concat([GL_flux['flux_Gt_ref_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ref_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ref_m5'].assign_coords({'m': 5})], dim='m')
GL_flux_ag['flux_Gt_ABUMIP'] = xr.concat([GL_flux['flux_Gt_ABUMIP_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ABUMIP_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ABUMIP_m5'].assign_coords({'m': 5})], dim='m')

GL_flux_ABUMIP = GL_flux_ag['flux_Gt_ABUMIP']

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_mask = file_isf['ISF_mask']

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [ ]:
mask_basins = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

In [ ]:
hf_limit = xr.open_dataset(inputpath_data + 'hydrofracturing_limits_new.nc')['hydrofrac_limit']

PREPARE DATA

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

sens_weights = sens_weights.drop_sel(model='GISS-E2-1-H')

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #'GISS-E2-1-H',

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])

In [ ]:
unviable_times = weighted_quantiles > 0
unviable_times_2300 = weighted_quantiles_2300 > 0

In [ ]:
viability_diff = unviable_times.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit_max = viability_diff.time.where(viability_diff == 0).max('time') + 1
limit_max = limit_max.where(limit_max<2099)
limit_max = limit_max.where(~(np.isnan(limit_max) & unviable_times.sel(time=1850)), 1850)

limit_max_full = limit_max.where(np.isfinite(limit_max), 2301)

unviable_isf = (unviable_times.time >= limit_max_full)
count_unviable_isf = (unviable_times.time >= limit_max_full).sum('Nisf')

In [ ]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2298,1849,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2299)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=1850)), 1850)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300)
count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')


In [ ]:
likelihood_list = ['vunlikely','unlikely','about','likely','vlikely']

In [ ]:
quant_distrib = unviable_isf.sel(quantile=[0,0.1,0.33,0.66,0.9,1]).sum('quantile')
quant_distrib_2300 = unviable_isf_2300.sel(quantile=[0,0.1,0.33,0.66,0.9,1]).sum('quantile')

In [ ]:
hf_doomed_2300 = (unviable_times_2300.time >= hf_limit)
quant_distrib_hydrofrac = hf_doomed_2300.sel(pctl=[1,10,33,66,90,99]).sum('pctl')

In [ ]:
count_hflimit_isf_2300 = (unviable_times_2300.time >= hf_limit).sum('Nisf')

In [ ]:
nonviable_before_hydrofrac = limit_max_full_2300.sel(quantile=0.33) <=  hf_limit.sel(pctl=33)

In [ ]:
nonviable_before_hydrofrac_clean = nonviable_before_hydrofrac.where((hf_limit.sel(pctl=33) < 2300) | (limit_max_full_2300.sel(quantile=0.33) <= 2297))

PREPARE PLOT

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
grounded_basin = mask_basins['Iceshelf'].where(grounded_msk == 0)

In [ ]:
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
### Colorbar:
cmap = mpl.cm.PuBu #Blues # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

alpha = 0.8
cmap_with_alpha = cmap(np.arange(cmap_new.N))
cmap_with_alpha[:, -1] = alpha
cmap_new3 = mpl.colors.ListedColormap(cmap_with_alpha)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new3.N)




In [ ]:
scen = 'ssp126'
yy = 2000
via_to_plot = quant_distrib_hydrofrac.sel(time=yy,scenario=scen) 
#via_to_plot = quant_distrib.sel(time=yy,scenario=scen)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']): #
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

fig = plt.figure(figsize=(8.25,8.25))

# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new3, norm=norm, levels=bounds)
plt.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new3, norm=norm, levels=bounds) #, alpha=0.7
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
plt.contour(grd.x,grd.y,file_isf['ISF_mask'],levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10) #.where(np.isnan(grounded_msk))
plt.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10)
plt.contour(grd.x,grd.y,grounded_msk,linewidths=0.7,colors='darkgrey',zorder=12)
plt.contour(grd.x,grd.y,icesheet_msk,linewidths=0.9,colors='black',zorder=15)

plt.title('Hydrofracturing probability '+str(yy)+' '+scen)

ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new3, norm=norm,
    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_hydrofracturing_'+str(yy)+'_'+scen+'_mod2300_withoutGISS.png', dpi=300)